# How to use the KMeans Inferrer

The inferenza KMeans model is desigend to mimic the basic predict method from SciKit Learn's [`KMeans` model class](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

To use the KMeans Inferrer follow these steps:
1. pretrain your Kmeans model using the `Kmeans` class from Scikit Learn
2. convert your model into ONNX format and save to an `.onnx` file
3. In your Rust project, load your model from the `.onnx` file and call the `predict` method to make predictions

## Step 1 - pretrain a KMeans model
Here we will use a very simple KMeans model as an example:

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

X, y, centers = make_blobs(n_samples=50, n_features=3, centers=5, cluster_std=0.5, shuffle=True, random_state=42,  return_centers=True)
num_features = X.shape[1]
num_clusters = centers.shape[0]

km = KMeans(n_clusters=num_clusters, init='k-means++', n_init=10, max_iter=500, tol=1e-04, random_state=42)
km.fit(X)

y_km = km.predict(X)



## Step 2 - save the model to `.onnx` file type

Here we use the `skl2onnx` package to convert our model into the `.onnx` file type so it can be injested by the inferenza KManes inferrer later on.

**A quick note about `initial_type`:**

Scikit-learn models are flexible in what they accept — for example, a KMeans model can take a NumPy array, a list of lists, or a DataFrame. However, ONNX is type-strict and needs to know the input signature of the model (i.e. the name, type and shape of the inputs you intend to use with this model later on).

The `initial_types` argument takes a list of tuple, with each tuple describing one input for the model. In this example we are only inclduing one input, so there is one tuple.

The first element in the tuple is the name of the input, here 'float_input'. This input name is only needed for the ONNX file structure but we do not call it internally during the inference process, meaning you can use any name you prefer.

The second argument in the tuple defines the shape and data type of the input. Here we use the `FloatTensorType` class provided by `skl2onnx` with our number of features as an argument, this describes a 2D float tensor of shape (batch_size, num_features). At this point we don't know what our batch size might be so we set the value to `None` to give us flexibility later.

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Output filename for the exported ONNX model
out_onnx = "kmeans.onnx"

# In this example, we are creating a float32 input of any batch size with 3 features
initial_type = [('float_input', FloatTensorType([None, num_features]))]

# Convert the trained scikit-learn KMeans model (km) to an ONNX model
onnx_model = convert_sklearn(km, initial_types=initial_type)

# Add custom metadata to the ONNX model for identification
meta =  onnx_model.metadata_props.add()
meta.key = "sklearn_model"
meta.value = "KMeans"

# Save the serialized ONNX model to disk
with open(out_onnx, "wb") as f:
    f.write( onnx_model.SerializeToString())


[('float_input', FloatTensorType(shape=[None, 3]))]


## Step 3 - Load your model and make predictions

Now that we've created an `.onnx` file with our pre-trained model, we can initialise the inferenza `KmeansModel` class and generate some predictions with unseen inputs.

```rust
// create an inferenza KmeansModel from your onnx file
let kmeans = KmeansModel::load_from_onnx_proto("<path_to_file>/kmeans.onnx");

// define your input as a 3x3 array (3 inputs with 3 features each)
let input = Array2::from_shape_vec(
    (3, 3), vec![
        3.9755416, -9.76483, 9.557824, // this should be predicted as class 0
        3.9755416, -9.76483, 9.557824, // this should be predicted as class 0
        -2.466838, 9.029932, 4.4263315 // this should be predicted as class 1
    ]
).unwrap();

// make predictions on our inputs with kmeans predict() function
let prediction = kmeans.predict(
    input
);

// print prediction result
println!("{:?}", prediction.to_vec());
// output: [0, 0, 1]

```

# Performance Comparison

The goal of the inferenza project is to provide a rust based inferrer that performs inferences faster than traditional python-based approaches. See below a speed comparison for making the preictions on the 3 inputs from the above example with SciKit Learn's built in predict function compared to inferenza:

**sklearn: 0.0012 seconds**  
**inferenza: 0.000095 seconds (~12x faster than sklearn)**

Test code for reference:

Python timed example

```python
import numpy as np
import time

x_test = np.array([
        [3.9755416, -9.76483, 9.557824],
        [3.9755416, -9.76483, 9.557824],
        [-2.466838, 9.029932, 4.4263315],
    ])

start = time.time()
y_km = km.predict(x_test)
end = time.time()

print(f"Elapsed time: {end - start:.4f} seconds")
```

Rust timed example
```rust
let input = Array2::from_shape_vec(
    (3, 3), vec![
        3.9755416, -9.76483, 9.557824, // 0
        3.9755416, -9.76483, 9.557824, // 0
        -2.466838, 9.029932, 4.4263315 // 1
    ]
).unwrap();

let start = Instant::now();
let prediction = kmeans.predict(
    input
);
let duration = start.elapsed();

let seconds = duration.as_secs_f64();

println!("Time elapsed: {:.6} seconds", seconds);
```